# Imports & Database

In [24]:
import pandas as pd
import logging
import os
from functions import post, parse, nuevos_avisos, guardar_avisos, descargar_pdf
# from NLP import extraer_texto_pdf, resumir_texto

In [25]:
palabras_clave = {'energía','energético'}

In [26]:
# Cargar la base de datos existente
database = pd.read_excel('database.xlsx')
database = database.drop(index=database.index)

# Logger

In [27]:
# Configurar logger
logging.basicConfig(
    filename='boletin_errors.log',
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [28]:
avisos_totales = []
for palabra in palabras_clave :
    try:
        response = post(palabra)
        if response.status_code == 200:
            datos = response.json()
            avisos_totales.extend(parse(datos))
        else:
            logging.warning(f"Error al buscar '{palabra}': Código {response.status_code}")
    except Exception as e:
        logging.error(f"Fallo la búsqueda para '{palabra}': {e}")

In [29]:
if avisos_totales is not None:
    df_nuevos_avisos = nuevos_avisos(avisos_totales, database)

In [30]:
# from transformers import pipeline
import fitz  # PyMuPDF
import logging


# Inicializar el modelo de resumen
# resumidor = pipeline("summarization", model="facebook/bart-large-cnn")

def extraer_texto_pdf(ruta_pdf):
    try:
        doc = fitz.open(ruta_pdf)
        texto = ""
        for pagina in doc:
            texto += pagina.get_text()
        return texto.strip()
    except Exception as e:
        logging.error(f"Error al extraer texto del PDF: {ruta_pdf}, Error: {e}")
        return ""

# def resumir_texto(texto, max_tokens=1024):
#     if not texto:
#         return ""
    
#     # Truncamos si el texto es demasiado largo
#     texto = texto[:4000]
    
#     try:
#         resumen = resumidor(texto, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
#         return resumen
#     except Exception as e:
#         logging.error(f"Error al resumir texto: {e}")
#         return ""


ModuleNotFoundError: No module named 'frontend'

In [ ]:
if not df_nuevos_avisos.empty:
    guardar_avisos(df_nuevos_avisos,database)
    for index, row in df_nuevos_avisos.iterrows():
        link_aviso = row['Link']
        resolucion = row['Resolución']

        ruta_completa = descargar_pdf(link_aviso,resolucion)

        texto = extraer_texto_pdf(ruta_completa)
        print(texto)
        # resumen = resumir_texto(texto)
        # df_avisos.at[index, "Resumen"] = resumen
